# Рекомендация тарифов

Есть данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта «Определение перспективного тарифа для телеком компании»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — Она сделана в предыдущем проекте.

Построить модель с максимально большим значением *accuracy*. Необходимо довести долю правильных ответов до 0.75. Проверить *accuracy* на тестовой выборке.

## Описание данных

- *сalls* — количество звонков
- *minutes* — суммарная длительность звонков в минутах
- *messages* — количество sms-сообщений
- *mb_used* — израсходованный интернет-трафик в Мб
- *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)

## Оглавление

1. [Открытие и изучение файла](#step1)
2. [Разбиение данных на выборки](#step2)
3. [Исследование модели](#step3)
4. [Проверка модели на тестовой сборке](#step4)
5. [Проверка модели на адекватность](#step5)
6. [Общий вывод](#step6)

<a id="step1"></a>
## 1. Открытие и изучение файла

In [1]:
# Подключение библиотек, необходимых модулей и алгоритмов
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

In [2]:
# прочитаем данные из файла и выведем первые 10 строк
users = pd.read_csv('users_behavior.csv')
users.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
# выведем названия столбцов таблицы
users.columns

Index(['calls', 'minutes', 'messages', 'mb_used', 'is_ultra'], dtype='object')

In [4]:
# выведем общую информацию по таблице
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод

На данном шаге проводилось первоначальное ознакомление с данными. Они были прочитаны из файла и сохранены в таблицу *users*. Была проведена базовая проверка данных на наличие пропусков, верного определения типов данных, правильного названия колонок в таблице. Все эти параметры определены правильно, ничего корректировать и исправлять не надо. Все готово для дальнейшей работы с таблицей.

<a id="step2"></a>
## 2. Разбиение данных на выборки

Для нашей задачи целевым признаком является признак `is_ultra`, который как раз и показывает, каким тарифом пользовался клиент в течение месяца. Выделим его в переменную `target`. Остальные признаки сохраним в переменной `features`.

In [5]:
features = users.drop('is_ultra', axis=1)
target = users['is_ultra']

Для работы с алгоритмами машинного обучения необходимо определить параметр рандомизации, который используется как начальное значение генератора псевдослучайных чисел. Чтобы зафиксировать одно определенное распределение при работе с данными, нужно использовать одно и то же значение пераметра `random_state`. Чтобы в дальнейшем не ошибиться при подстановке выбранного числа в методах для машинного обучения, сохраним его в переменной `rand_seed`, и в дальнейшем будем использовать его.

In [6]:
rand_seed = 23

Теперь необходимо разделить все данные на обучающую, валидационную и тестовые выборки. В нашей задаче спрятанной тестовой выборки нет, поэтому разделим исходные данные в отношении 3:1:1. Доли валидационной и тестовой выборок будут равны и составлять по 20%. Поскольку функция `train_test_split` делит только на 2 выборки, мы разделим данные в 2 этапа - сначала 3:2,а затем вторую часть данных пополам.

In [7]:
# отделим 40% данных для проверочных выборок
features_train, features_check, target_train, target_check = train_test_split(
    features, target, test_size=0.4, random_state=rand_seed)

In [8]:
# определим 50% данных для валидационной выборки и 50% для тестовой выборки (из проверочной выборки)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_check, target_check, test_size=0.5, random_state=rand_seed)

In [9]:
# Проверим, что данные распределены правильно
print('Training:')
print('\t features shape:', features_train.shape)
print('\t target shape:', target_train.shape)

print('Validation:')
print('\t features shape:', features_valid.shape)
print('\t target shape:', target_valid.shape)

print('Testing:')
print('\t features shape:', features_test.shape)
print('\t target shape:', target_test.shape)

Training:
	 features shape: (1928, 4)
	 target shape: (1928,)
Validation:
	 features shape: (643, 4)
	 target shape: (643,)
Testing:
	 features shape: (643, 4)
	 target shape: (643,)


### Вывод

На данном шаге проводилось разбиение данных на выборки: обучающую, валидационную и тестовую. Выборки были разделены в отношении 3:1:1, таким образом размеры выборок соотносятся 1928:643:643.

<a id="step3"></a>
## 3. Исследование модели

На данном шаге необходимо исследовать различные модели с разными гиперпараметрами и выбрать ту, у которой доля правильных ответов будет наибольшей. Стоит задачи бинарной классификации. В качества моделей будем рассматривать решающее дерево, случайный лес и логистическую регрессию.

Начнем с решающего дерева. В качестве исследуемых гиперпараметров возьмем максимальную глубину дерева (`max_depth`) в диапазоне от 1 до 10, минимальное количество примеров для разделения (`min_samples_split`) в диапазоне от 2 до 10, минимальное количество объектов в листе (`min_samples_leaf`) в диапазоне от 1 до 10. Сохраним модель и ее гиперпараметры для самой высокой доли правильный ответов.

In [10]:
# Определим первоначальные значения
best_dtc_accuracy = 0
best_dtc_model = None
best_dtc_depth = 1
best_dtc_samples_split = 2
best_dtc_samles_leaf = 1

# Проведем исследование
for depth in range(1, 11):
    for samples_split in range(2, 11):
        for samles_leaf in range(1, 11):
            model = DecisionTreeClassifier(
                random_state=rand_seed, 
                max_depth=depth, 
                min_samples_split=samples_split, 
                min_samples_leaf=samles_leaf
            )
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if accuracy > best_dtc_accuracy:
                best_dtc_model = model
                best_dtc_accuracy = accuracy
                best_dtc_depth = depth
                best_dtc_samples_split = samples_split
                best_dtc_samles_leaf = samles_leaf
                
print('best_dtc_accuracy =', best_dtc_accuracy)
print('best_dtc_depth =', best_dtc_depth)
print('best_dtc_samples_split =', best_dtc_samples_split)
print('best_dtc_samles_leaf =', best_dtc_samles_leaf)

best_dtc_accuracy = 0.8009331259720062
best_dtc_depth = 8
best_dtc_samples_split = 2
best_dtc_samles_leaf = 3


Далее рассмотрим модель случайного леса. Здесь, в качестве гиперпараметров для исследования возьмем те же, что и для решающего дерева: `max_depth`, `min_samples_split`, `min_samples_leaf`. Также добавим исследование по количеству оценщиков (`n_estimators`), который будет изменяться в диапазоне от 2 до 10.

In [11]:
# Определим первоначальные значения
best_rfc_accuracy = 0
best_rfc_model = None
best_rfc_depth = 1
best_rfc_samples_split = 2
best_rfc_samles_leaf = 1
best_rfc_estimators = 2

# Проведем исследование
for depth in range(1, 11):
    for samples_split in range(2, 11):
        for samles_leaf in range(1, 11):
            for estimators in range(2, 11):
                model = RandomForestClassifier(
                    random_state=rand_seed, 
                    max_depth=depth, 
                    min_samples_split=samples_split, 
                    min_samples_leaf=samles_leaf,
                    n_estimators=estimators
                )
                model.fit(features_train, target_train)
                accuracy = model.score(features_valid, target_valid)
                if accuracy > best_rfc_accuracy:
                    best_rfc_model = model
                    best_rfc_accuracy = accuracy
                    best_rfc_depth = depth
                    best_rfc_samples_split = samples_split
                    best_rfc_samles_leaf = samles_leaf
                    best_rfc_estimators = estimators
                
print('best_rfc_accuracy =', best_rfc_accuracy)
print('best_rfc_depth =', best_rfc_depth)
print('best_rfc_samples_split =', best_rfc_samples_split)
print('best_rfc_samles_leaf =', best_rfc_samles_leaf)
print('best_rfc_estimators =', best_rfc_estimators)

best_rfc_accuracy = 0.8118195956454122
best_rfc_depth = 9
best_rfc_samples_split = 9
best_rfc_samles_leaf = 3
best_rfc_estimators = 10


Рассмотрим также модель логистической регрессии. В данном случае рассмотрим работу модели для гиперпараметров по умолчанию.

In [12]:
# Определим первоначальные значения
best_lr_accuracy = 0
best_lr_model = None

# Проведем исследование
    # solver='lbfgs' - указано, чтобы скрыть предупреждение об изменении значения solver по умолчанию
model = LogisticRegression(random_state=rand_seed, solver='lbfgs')
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
if accuracy > best_lr_accuracy:
    best_lr_model = model
    best_lr_accuracy = accuracy
                
print('best_lr_accuracy =', best_lr_accuracy)

best_lr_accuracy = 0.6780715396578538


### Вывод

На данном шаге были исследованы 3 модели для обучения. Доля правильных ответов выше всего получилась для модели случайного леса - 81%. Такое значение получилось для гиперпараметров: `max_depth`=9, `min_samples_split`=9, `min_samples_leaf`=3, `n_estimators`=10. При этом n_estimators бралось в небольшом диапазоне (до 10), по причине того, что проверка всех возможных вариантов занимает слишком много времени. Для решающего дерева доля правильных ответов получилась не сильно меньше - 80%. Но если мы получили результат лучше для первой модели за разумное время, лучше использовать первую модель для тестовых данных. Для логистической регрессии значение правильности получилось не столь высоким. Это связано с тем, что для нее не так подробно исследовались гиперпараметры.

<a id="step4"></a>
## 4. Проверка модели на тестовой выборке

На данном шаге проверим работу модели, с наибольшим значением доли правильных ответов для валидационной выборки, на тестовых данных. Как мы выяснили на предыдущем шаге, такой моделью является модель случайного леса. Соответствующие гиперепараметры сохранены при исследовании.

In [13]:
model = RandomForestClassifier(
        random_state=rand_seed, 
        max_depth=best_rfc_depth, 
        min_samples_split=best_rfc_samples_split, 
        min_samples_leaf=best_rfc_samles_leaf,
        n_estimators=best_rfc_estimators
)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Accuracy =", accuracy)

Accuracy = 0.8227060653188181


### Вывод

Как видно из этого шага, для тестовой выборки значение доли правильных ответов получилось почти такое же (0,8227), как и для валидационной (0,8118). Это означает, что мы правильно подобрали гиперпараметры, и эту модель можно использовать в дальнейшем для предсказания тарифа по данным о поведении клиентов.

<a id="step5"></a>
## 5. Проверка модели на адекватность

На данном шаге проверим модель на вменяемость. Для этого узнаем сколько клиентов использует каждый тариф.

In [14]:
target_train.value_counts()

0    1332
1     596
Name: is_ultra, dtype: int64

Видно, что большинство клиентов использует тариф smart (потому что значению параметра is_ultra == 0 соответствует 1332 записи против 596 для is_ultra == 1). Обучим модель таким образом, чтобы она всегда предсказывала, что тариф будет smart.

In [15]:
check_predictions = pd.Series(0, index=target_test.index)
check_accuracy = accuracy_score(target_test, check_predictions)
print("Check accuracy =", check_accuracy)

Check accuracy = 0.7153965785381027


### Вывод

На данном шаге мы проверяли модель на адекватность. Для этого мы сравнили с ситуацией, когда модель всегда предсказывала бы что необходимо выбрать тариф smart. Как видно из результата, доля правильных ответов в данном случае соствляет 0,7154. Полученное значение для модели случайного леса выше, что показывает на эффективность выбранного решения.

<a id="step6"></a>
## 6. Общий вывод

В данном проекте проводилась бинарная классификация для выбора между двумя тарифами по данным о клиентах, которые уже используют их. Для выполнения этой задачи, первоначальная выборка была разбита на обучающую, валидационную и тестовую. Далее были определены параметры и целевой параметр. После чего проводилось исследование нескольких моделей с разными значениями гиперпараметров. При изучении были рассмотрены модели решающего дерева, случайного леса и логистической регрессии. В итоге была выбрана модель случайного леса для определенного набора гиперпараметров, как модель с самой высокой долей правильных ответов - 0,81. После чего, данная модель была проверена на тестовой сборке, правильность составила более 0.75, что является успешным результатом. После этого, модель была проверена на вменяемость, и рассмотрен случай, как если бы она всегда предсказывала только один тариф. Доля правильных ответов для данного случая составила 0,72, это показывает, что полученная в ходе иследования модель - адекватна.